<a href="https://colab.research.google.com/github/Trongnhat191/Yeast-Cell-Detection/blob/main/Get_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/MyDrive/Yeast_Nhat

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import tensorflow as tf

In [ ]:
!pip install tensorflow==2.15.0

In [ ]:
def cut_unecessary_img(head_path,input_paths,save_paths):
  for i in range(len(input_paths)):
    image = cv2.imread(os.path.join(head_path, input_paths[i]))
    cv2_imshow(image)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    threshold_value = 185
    ret, thresholded_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

    thresholded_image = cv2.bitwise_not(thresholded_image)

    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(gray_image)

    new_contours = ()

    MIN_HEIGHT = image.shape[1] * 0.5
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if h >= MIN_HEIGHT:
            new_contours += (cnt,)
    con = new_contours[0]
    cv2.drawContours(mask, [con], -1, (255), thickness=cv2.FILLED)
    x ,y,w,h = cv2.boundingRect(con)
    # Apply the mask to the original image
    result = cv2.bitwise_and(image, image, mask=mask)
    # result = resize_image(result)
    # result = pad_img(result, (1536, 1536))
    cv2_imshow(result)
    result = result[y:y+1536,x:x+1536]
    cv2_imshow(result)
    plt.show()
    # Display the result
    save_path = os.path.join(save_paths,input_paths[i])
    # cv2.imwrite(save_path, result)


def pad_img(img, target_size = (1536, 1536)):
    # Lấy kích thước hiện tại của ảnh
    h, w = img.shape[:2]

    # Tính toán số lượng padding cần thiết
    pad_h = max(0, (target_size[0] - h) // 2)
    pad_w = max(0, (target_size[1] - w) // 2)

    ex1 = (target_size[0] - h) % 2
    ex2 = (target_size[1] - w) % 2
    # Pad ảnh sao cho kích thước đạt target_size
    padded_img = cv2.copyMakeBorder(
        img,
        pad_h, pad_h + ex1, pad_w, pad_w +ex2,
        borderType=cv2.BORDER_CONSTANT,
        value=0
    )

    # Cắt ảnh nếu kích thước vượt quá target_size sau padding
    padded_img = padded_img[:target_size[0], :target_size[1]]

    return padded_img

In [ ]:
images_2h_list = os.listdir('Back_test/F1_B1_2H')
images_4h_list = os.listdir('Back_test/F1_B1_4H')
cut_unecessary_img('Back_test/F1_B1_2H',images_2h_list,'Back_test_Cut_Outside/F1_B1_2H' )
cut_unecessary_img('Back_test/F1_B1_2H',images_2h_list,'Back_test_Cut_Outside/F1_B1_2H' )

In [ ]:
images_4h_list = os.listdir('Back_test_Cut_Outside/F1_B1_4H')
images_2h_list = os.listdir('Back_test_Cut_Outside/F1_B1_2H')
print(len(images_4h_list))
for i in images_4h_list:
  tmp =cv2.imread(os.path.join('Back_test_Cut_Outside/F1_B1_4H',i))
  tmp = pad_img(tmp, (1536, 1536))
#   cv2.imwrite(os.path.join('Back_test_Cut_Outside/F1_B1_4H',i), tmp)
  print(tmp.shape)

for i in images_2h_list:
  tmp =cv2.imread(os.path.join('Back_test_Cut_Outside/F1_B1_2H',i))
  tmp = pad_img(tmp, (1536, 1536))
#   cv2.imwrite(os.path.join('Back_test_Cut_Outside/F1_B1_2H',i), tmp)
  cv2_imshow(tmp)


In [ ]:
def read_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    # Convert the TensorFlow tensor to a NumPy array
    image = image.numpy() # Add this line to convert to NumPy array
    image = pad_img(image, (1536, 1536))
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [ ]:
two_folders_path_list = os.listdir('Back_test')
print(two_folders_path_list)
images_4h_list = os.listdir('Back_test/F1_B1_4H')
print(len(images_4h_list))
images_2h_list = os.listdir('Back_test/F1_B1_2H')
print(len(images_2h_list))

In [ ]:
images = []

for f in images_2h_list:
    image_path = os.path.join('Back_test_Cut_Outside/F1_B1_2H', f)
    image = read_image(image_path)
    images.append(image)


In [ ]:
images.shape

In [ ]:
images_path_list = os.listdir('Tu_Lieu_AI_Goc_Cut_Outside')
for ipl in images_path_list:
  print(ipl)
  image = cv2.imread(os.path.join('Tu_Lieu_AI_Goc_Cut_Outside',ipl))
  image = pad_img(image, (1536, 1536))
  cv2.imwrite(os.path.join("train_data/images", ipl), image)

In [ ]:
from keras.models import load_model

unet = load_model("68unet-non-aug.h5")

In [ ]:
output_dir = "mask"
# os.makedirs(output_dir, exist_ok=True)
batch_size = 1
predictions = []
for i in range(0, len(images), batch_size):
    batch_predictions = unet.predict(images[i:i+batch_size])
    predictions.extend(batch_predictions)

predictions = np.array(predictions)

# Hiển thị và lưu các ảnh và mask dự đoán tương ứng
for i in range(len(images_path_list)):
    plt.figure(figsize=(20, 10))

    # Ảnh gốc
    plt.subplot(1, 2, 1)
    plt.imshow(images[i])
    plt.title("Original Image")

    # Mask dự đoán
    plt.subplot(1, 2, 2)
    plt.imshow(predictions[i].squeeze(), cmap='gray')
    output_file = os.path.join(output_dir, images_path_list[i])
    plt.imsave(output_file, predictions[i].squeeze(), cmap='gray')
    plt.title("Predicted Mask")

    plt.show()
